# Signal Spaces

## Basic data types

In [1]:
from typing import NewType, Iterable, Sequence, Callable, TypeVar, Generic, Union, Generator, Any

In [2]:
SigType=float

In [3]:
Sig_0=NewType('Sig_0',tuple[SigType])
Sig_1=NewType('Sig_1',tuple[Sig_0])
Sig_2=NewType('Sig_2',tuple[Sig_0,Sig_0])
TripleBlocks=NewType('TripleBlocks',tuple[Sig_0,Sig_0,Sig_0])
Sig_3=NewType('Sig_3',tuple[TripleBlocks])

In [4]:
Vector = list[float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]

In [5]:
new_vector = scale(2.0, [1.0, -4.2, 5.4])
print(new_vector)

[2.0, -8.4, 10.8]


In [6]:
a=(1,2,3,4)
a_s=Sig_0(tuple(a))    

In [7]:
b=(Sig_0(tuple((1,2))),Sig_0(tuple((3,4))),Sig_0(tuple((5,6))),Sig_0(tuple((7,8))))
b_s=Sig_1(tuple(b))
print(b_s)

((1, 2), (3, 4), (5, 6), (7, 8))


In [8]:
c=TripleBlocks(tuple((a_s,a_s,a_s)))
Sig_3(tuple((c,)))

(((1, 2, 3, 4), (1, 2, 3, 4), (1, 2, 3, 4)),)

python compiler does not check type in run time, so the above is just type hints. We need to use a static type checker to check the above type definitions and their applications.

In vscode, to use the static type checker, put the following lines into the `.vscode/settings.json`. After installing the vscode extension `Pylance` (by microsoft), the type checker will do the checking for the file opened with the editor.
```json
{
  "python.analysis.typeCheckingMode": "basic"
}
```

In [9]:
a=Sig_0([1,2,3,4])
b=Sig_0([5,6,7,8])
Sig_3([a,b,a])

[[1, 2, 3, 4], [5, 6, 7, 8], [1, 2, 3, 4]]

In [10]:
a=Sig_0((1,2,3,4))
b=Sig_0([5,6,7,8])
Sig_3((a,b,a,b))

((1, 2, 3, 4), [5, 6, 7, 8], (1, 2, 3, 4), [5, 6, 7, 8])

### Operators on the signal data structure
These functions are hardly used, instead, we usually use index slicing directly. But here we illustrate the concepts.

In [11]:
def head(s:Sequence[Any])->Any:
    return s[0]

In [12]:
def tail(s:Sequence[Any])->Sequence[Any]:
    return s[1:]

In [13]:
def last(s:Sequence[Any])->Any:
    return s[-1]

In [14]:
def take(n:int,s:Sequence[Any])->Sequence[Any]:
    return s[:n]

In [15]:
def drop(n:int,s:Sequence[Any])->Sequence[Any]:
    return s[n:]

In [16]:
def length(s:Sequence[Any])->int:
    if s==():
        return 0
    else:
        return 1+length(drop(1,s))

In [17]:
length(a)

4

### foldl and foldr

In [18]:
B=TypeVar("B")
A=TypeVar("A")
T=Generic[A]

In [19]:
def foldl(f: Callable[[B, A], B], x0: B, v: Iterable[A]) -> Generator[B,None,None]:
    for i in v:
        r = f(x0, i)
        x0 = r
        yield r

In [20]:
def foldr(f:Callable[[B,A],B],x0:B,v:Sequence[A])->Generator[B,None,None]:
    for i in range(1,1+len(v)):
        r = f(x0,v[-i])
        x0=r
        yield r

In [21]:
foo=foldl((lambda acc,x: acc+x),0,[1,2,3,4])

In [22]:
for i in range(3):
    print(next(foo))

1
3
6


In [23]:
foo=foldl((lambda acc,x: (acc[1][-2:],x)),(None,(-1,0)),((1,2,3,4),(5,6,7,8)))

In [24]:
for i in foo:
    print(i)

((-1, 0), (1, 2, 3, 4))
((3, 4), (5, 6, 7, 8))


In [25]:
foo=foldr((lambda acc,x:(x,acc[0][:2])),((9,10),None),((1,2,3,4),(5,6,7,8)))
for i in foo:
    print(i)

((5, 6, 7, 8), (9, 10))
((1, 2, 3, 4), (5, 6))


## Basic morphisms

### blocks: `Sig_0` to `Sig_1` morphism

In [26]:
def blocks(n:int,s:Sig_0)->Sig_1:
    if n<=0:
        return Sig_1((s,))
    elif s==Sig_0(tuple()):
        return  () #Sig_1(tuple((Sig_0(tuple()),)))
    return (s[:n],)+blocks(n,s[n:]) # (take(n,s),)+blocks(n,drop(n,s))

In [27]:
a=Sig_0(tuple(x for x in range(8)))
blocks(2,a)

((0, 1), (2, 3), (4, 5), (6, 7))

In [28]:
blocks(0,a)

((0, 1, 2, 3, 4, 5, 6, 7),)

In [29]:
blocks(len(a),a)

((0, 1, 2, 3, 4, 5, 6, 7),)

### oblocks: `Sig_1` to `Sig_3` morphism

In [30]:
foo = blocks(2,a)
print(foo)

((0, 1), (2, 3), (4, 5), (6, 7))


In [31]:
list(map(sum,foo))

[1, 5, 9, 13]